In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Read the CSV data file into a Pandas DataFrame object
data = pd.read_csv('B_010.CSV', delimiter=';', header=None)
df = data.replace(r'^\s*$', np.nan, regex=True).astype(float)   # df contains empty cells, 
                                                                # replace them with "NaN"
# Print the updated DataFrame
print(df.head(), "\n", df.tail())   # First and last 5 rows.

        0             1           2          3          4       5      6    7  \
0     NaN           NaN         NaN        NaN        NaN     NaN    1.0  0.0   
1  5010.0  9.926921e+00      0.0000     6.0000   -1.00000   1.000  525.0  1.0   
2   293.6  0.000000e+00    421.0000     0.0000  430.00000   0.000  525.0  1.0   
3     0.0  1.000000e+10  10000.0000  1000.0000  100.00000  10.000  525.0  1.0   
4     1.0  1.000000e-05      0.0001     0.0005    0.00075   0.001  525.0  1.0   

       8    9  
0    0.0  0.0  
1  451.0  1.0  
2  451.0  2.0  
3  451.0  3.0  
4  451.0  4.0   
                0    1    2    3    4    5      6    7      8        9
355337  0.000002  0.0  0.0  0.0  0.0  0.0  525.0  6.0  221.0   7961.0
355338  0.000000  0.0  NaN  NaN  NaN  NaN  525.0  6.0  221.0   7962.0
355339       NaN  NaN  NaN  NaN  NaN  NaN  525.0  6.0    0.0  99999.0
355340       NaN  NaN  NaN  NaN  NaN  NaN    0.0  0.0    0.0      0.0
355341       NaN  NaN  NaN  NaN  NaN  NaN   -1.0  0.0    0.0     

In [3]:
"""
=======================================================
The purpose of this is function not 100% clear, but 
it plays an important role for "extract_mf3" and 
"extract_mf6" functions for extracting data from 
the ENDF file.
-------------------------------------------------------
INPUT:
n - integer; the total number of words that need to be 
    extracted from the m matrix (whatever that means);
iRow - integer; the index of the starting row of 
       the "m" matrix;
m - 2D NumPy array; essentially a matrix containing 
    the data to be extracted.
-------------------------------------------------------
NOTE: Why "m" is a NumPy array, not a Pandas DataFrame?
-------------------------------------------------------
NumPy is usually much more forgiving if you are trying 
to access an element in an array. Essentially:

        a[x,y] = a[x][y].

For a Pandas DF, you would have to use .iloc[]
which gets very bothersome, very quicly. Also,
NumPy is just faster. Simple as that.
-------------------------------------------------------
OUTPUT:
a - 1D NumPy array;
iRowNew - integer; updated row number
=======================================================
"""
def extractNwords(n, iRow, m):
    k = 0
    iRowNew = iRow
    a = np.empty((1,n), dtype=np.float64)
    for ii in range(int(n/6)):  # read lines with 6 words each
        for jj in range(6):
            a[0][k] = m[iRowNew][jj]
            k += 1
        iRowNew += 1

    if (n - int(n/6)*6) == 0:
        iRowNew -= 1    

    for jj in range((n-int(n/6)*6)):  # read the last line with less than 6 words
        a[0][k] = m[iRowNew][jj]
        k += 1

    return a, iRowNew

In [4]:
"""
==========================================================
In an ENDF type of file, MF 3 refers to the section that 
provides the cross-sections for outgoing neutrons produced 
by a particular reaction. Specifically, MF 3 contains the 
angular distributions and energy distributions of the 
outgoing neutrons, which are necessary for determining 
how the neutrons will interact with matter in subsequent 
calculations. The data in MF 3 is typically provided in 
tabular form as a function of incident neutron energy and 
scattering angle. This function extracts this data from
the preprocessed CSV file.
----------------------------------------------------------
INPUT:
mt  - integer; reaction
ntt - integer; index for temperature value
m - 2D NumPy array;
----------------------------------------------------------
OUTPUT:
sig(nSig0, enGroup)- 2D NumPy array; where "enGroup" is 
the number of energy groups and "nSig0" is the the number 
of sigma-zeros.
==========================================================
"""
def extract_mf3(mt, ntt, m):
    nRow = m.shape[0]  # number of rows
    nTemp = 0  # number of temperatures
    iRowFound = 0
    for iRow in range(nRow):
        #if m.iloc[iRow, 7] == 3 and m.iloc[iRow, 8] == mt and m.iloc[iRow, 9] == 1:
        if m[iRow, 7] == 3 and m[iRow, 8] == mt and m[iRow, 9] == 1:
            # find the row with mf=3 and required mt
            nTemp += 1  # number of temperatures
            if nTemp == ntt:
                iRowFound = iRow + 1
                break
    if iRowFound > 0:  # there is mf=3 and required mt for this isotope
        nSig0 = int(m[iRowFound-1, 3])  # number of sigma-zeros
        nLgn = int(m[iRowFound-1, 2])  # number of Legendre components
        iRow = iRowFound + 1
        enGroup = int(m[2, 2])
        sig = np.zeros((nSig0, enGroup))
        while m[iRow, 7] == 3 and m[iRow, 8] == mt:
            ig = int(m[iRow-1, 5])
            a, iRowNew = extractNwords(nSig0 * nLgn * 2, iRow, m)
            sig[0:nSig0, ig-1] = a[0, nSig0*nLgn:(nSig0*nLgn+nSig0)] # Benefits of using NumPy:
            iRow = iRowNew + 2                                       # a[x,y] = a[x][y]
    else:
        sig = 0
    return sig

# Example to check against MATLAB
mt = 102  # set the value of mt
iTemp = 1  # set the value of ntt
m = df.to_numpy()  # convert DataFrame to NumPy array
sig = extract_mf3(mt, iTemp, m)  # apply the function to the numpy array
sig

array([[10.42469 ,  4.513115,  3.177173, ...,  0.      ,  0.      ,
         0.      ],
       [ 9.954398,  4.388016,  3.169668, ...,  0.      ,  0.      ,
         0.      ],
       [ 9.914734,  4.361569,  3.167052, ...,  0.      ,  0.      ,
         0.      ],
       [ 9.910362,  4.358269,  3.166685, ...,  0.      ,  0.      ,
         0.      ],
       [ 9.909923,  4.357931,  3.166647, ...,  0.      ,  0.      ,
         0.      ],
       [ 9.909876,  4.357897,  3.166643, ...,  0.      ,  0.      ,
         0.      ]])

In [ ]:
"""
NOTE: Work in progress! 
==============================================================
MF 6 stands for "Cross Sections" in an ENDF (Evaluated Nuclear
Data File) type of file. This section contains the evaluated 
cross-section data, such as the neutron-induced reaction cross 
sections, as a function of incident neutron energy. The cross-
section data may be provided in tabular or in resonance format. 
Additionally, the section may also contain the angular distri-
butions, energy-angle distributions, or other related data. 
This information is essential for the calculation of the neut-
ron transport in nuclear reactors and other applications.
--------------------------------------------------------------
INPUT:
mt  - integer; reaction;
ntt - integer; index for temperature value;
m - 2D NumPy array.
--------------------------------------------------------------
OUTPUT:
ifrom - 1D NumPy array;
ito   - 1D NumPy array;
sig   - dictionary (maybe?); list of dimensional data
==============================================================
"""
def extract_mf6(mt, ntt, m):
    iRow = 0 # row number
    nTemp = 0 # number of temperatures
    ifrom = np.array([], dtype=np.float64) # index of group 'from'
    ito = np.array([], dtype=np.float64) # index of group 'to'
    sig = {} # dictionary to store the data
    
    while m[iRow,6] != -1: # up to the end
        if m[iRow,7] == 6 and m[iRow,8] == mt: # find the row with mf=6 & mt
            if m[iRow,9] == 1: # this is the first line of mf=6 & mt: initialize
                nonz = 0 # number of nonzeros
                nLgn = m[iRow,2] # number of Legendre components
                nSig0 = m[iRow,3] # number of sigma-zeros

                iRow += 1
                nTemp += 1 # temperature index
            ng2 = m[iRow,2] # number of secondary positions
            ig2lo = m[iRow,3] # index to lowest nonzero group
            nw = m[iRow,4] # number of words to be read
            ig = m[iRow,5] # current group index

            iRow += 1     
            a, iRowNew = extractNwords(nw, iRow, m) # extract nw words in vector a
            iRow = iRowNew

            if nTemp == ntt:
                k = nLgn*nSig0 # the first nLgn*nSig0 words are flux -- skip.
                for iTo in range(ig2lo, ig2lo+ng2-1):
                    nonz += 1
                    ifrom.append(ig)
                    ito.append(iTo)
                    for iSig0 in range(1, nSig0+1):
                        for iLgn in range(1, nLgn+1):
                            k += 1
                            # TODO: save dimesion info to "sig"

            iRow += 1
    if nTemp == 0:
        sig = 0

ifrom2, ito2, sig2 = extract_mf6(16, iTemp, m)